In [1]:
print('hi')

hi


In [2]:
from IPython.core.display import HTML
HTML("<script>Jupyter.notebook.kernel.restart()</script>")

#restart kernel

from IPython.core.display import HTML
HTML("<script>Jupyter.notebook.kernel.restart()</script>")

#restart kernel

In [3]:
# ===== INSTALL ONLY (NO IMPORTS HERE) =====
import os, sys, subprocess

def pip_install(pkg):
    subprocess.check_call([sys.executable, "-m", "pip", "install", "-q", pkg])

# Disable MediaPipe Tasks
os.environ["MEDIAPIPE_DISABLE_TASKS"] = "1"

# Install NumPy-2-compatible OpenCV and MediaPipe
pip_install("opencv-python>=4.10.0")
pip_install("mediapipe==0.10.13")
pip_install("tqdm")

print("INSTALL DONE — NOW RESTART KERNEL")


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 35.6/35.6 MB 51.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 294.9/294.9 kB 15.8 MB/s eta 0:00:00


ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
bigframes 2.26.0 requires google-cloud-bigquery-storage<3.0.0,>=2.30.0, which is not installed.
grain 0.2.15 requires protobuf>=5.28.3, but you have protobuf 4.25.8 which is incompatible.
ray 2.52.1 requires click!=8.3.*,>=7.0, but you have click 8.3.1 which is incompatible.
opentelemetry-proto 1.37.0 requires protobuf<7.0,>=5.0, but you have protobuf 4.25.8 which is incompatible.
ydf 0.13.0 requires protobuf<7.0.0,>=5.29.1, but you have protobuf 4.25.8 which is incompatible.
bigframes 2.26.0 requires rich<14,>=12.4.4, but you have rich 14.2.0 which is incompatible.
grpcio-status 1.71.2 requires protobuf<6.0dev,>=5.26.1, but you have protobuf 4.25.8 which is incompatible.
gcsfs 2025.3.0 requires fsspec==2025.3.0, but you have fsspec 2025.10.0 which is incompatible.


INSTALL DONE — NOW RESTART KERNEL


In [4]:
from IPython.core.display import HTML
HTML("<script>Jupyter.notebook.kernel.restart()</script>")

#restart kernel

In [5]:
import numpy as np
import cv2
import mediapipe as mp

print("NumPy:", np.__version__)
print("OpenCV:", cv2.__version__)
print("MediaPipe:", mp.__version__)
print("Has solutions:", hasattr(mp, "solutions"))


2025-12-19 07:03:43.410036: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:467] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1766127823.659150      55 cuda_dnn.cc:8579] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1766127823.728874      55 cuda_blas.cc:1407] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
W0000 00:00:1766127824.293467      55 computation_placer.cc:177] computation placer already registered. Please check linkage and avoid linking the same target more than once.
W0000 00:00:1766127824.293518      55 computation_placer.cc:177] computation placer already registered. Please check linkage and avoid linking the same target more than once.
W0000 00:00:1766127824.293521      55 computation_placer.cc:177] computation placer alr

NumPy: 2.0.2
OpenCV: 4.12.0
MediaPipe: 0.10.13
Has solutions: True


In [6]:
import mediapipe as mp

mp_holistic = mp.solutions.holistic

holistic = mp_holistic.Holistic(
    static_image_mode=False,
    model_complexity=1,
    smooth_landmarks=True,
    refine_face_landmarks=False,
    min_detection_confidence=0.5,
    min_tracking_confidence=0.5
)

print("Holistic ready")


Holistic ready


INFO: Created TensorFlow Lite XNNPACK delegate for CPU.


In [7]:
import numpy as np

def extract_258d_landmarks(results):
    feat = []

    # Pose (33 × 3 = 99)
    if results.pose_landmarks:
        for lm in results.pose_landmarks.landmark:
            feat.extend([lm.x, lm.y, lm.z])
    else:
        feat.extend([0.0] * 99)

    # Left hand (21 × 3 = 63)
    if results.left_hand_landmarks:
        for lm in results.left_hand_landmarks.landmark:
            feat.extend([lm.x, lm.y, lm.z])
    else:
        feat.extend([0.0] * 63)

    # Right hand (21 × 3 = 63)
    if results.right_hand_landmarks:
        for lm in results.right_hand_landmarks.landmark:
            feat.extend([lm.x, lm.y, lm.z])
    else:
        feat.extend([0.0] * 63)

    # Face (11 selected points × 3 = 33)
    FACE_IDX = [1, 33, 61, 199, 263, 291, 10, 152, 234, 454, 356]
    if results.face_landmarks:
        for idx in FACE_IDX:
            lm = results.face_landmarks.landmark[idx]
            feat.extend([lm.x, lm.y, lm.z])
    else:
        feat.extend([0.0] * 33)

    return np.array(feat, dtype=np.float32)


In [8]:
import cv2

def process_video_to_tensor(video_path, target_frames=30):
    cap = cv2.VideoCapture(video_path)
    frames = []

    while True:
        ret, frame = cap.read()
        if not ret:
            break
        frame = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
        results = holistic.process(frame)
        frames.append(extract_258d_landmarks(results))

    cap.release()

    if len(frames) == 0:
        return None

    # Resample to 30 frames
    idxs = np.linspace(0, len(frames) - 1, target_frames).astype(int)
    frames = [frames[i] for i in idxs]

    return np.stack(frames)  # (30, 258)


In [9]:
import json, os

OUT_ROOT = "/kaggle/working/preprocessed"
os.makedirs(OUT_ROOT, exist_ok=True)

INDEX_PATH = f"{OUT_ROOT}/processed_index.json"

if os.path.exists(INDEX_PATH):
    processed = json.load(open(INDEX_PATH))
else:
    processed = {}

def mark_done(word):
    processed[word] = True
    with open(INDEX_PATH, "w") as f:
        json.dump(processed, f, indent=2)


In [ ]:
from glob import glob
from tqdm import tqdm
import os

DATA_ROOTS = [
    "/kaggle/input/bdslw401/Front/Front"
]

for wid in range(1, 301):
    word = f"W{wid:03d}"

    if processed.get(word):
        continue

    print(f"\nProcessing {word}")
    out_dir = f"{OUT_ROOT}/{word}"
    os.makedirs(out_dir, exist_ok=True)

    videos = []
    for root in DATA_ROOTS:
        videos += glob(f"{root}/**/{word}*.mp4", recursive=True)

    for vpath in tqdm(videos):
        fname = os.path.basename(vpath).replace(".mp4", ".npy")
        out_path = f"{out_dir}/{fname}"

        if os.path.exists(out_path):
            continue

        tensor = process_video_to_tensor(vpath)
        if tensor is None:
            continue

        np.save(out_path, tensor)

    mark_done(word)
